#### 0. Data Exploration and Cleaning

In [2]:
# read data
df = spark.read.load("/FileStore/tables/animals_comments_csv-5aaff.gz", format='csv', header = True, inferSchema = True)
from pyspark.sql.types import IntegerType
df = df.withColumn("userid", df["userid"].cast(IntegerType()))
df.show(10)

+--------------------+------+--------------------+
 creator_name|userid| comment|
+--------------------+------+--------------------+
 Doug The Pug| 87|I shared this to ...|
 Doug The Pug| 87| Super cute 😀🐕🐶|
 bulletproof| 530|stop saying get e...|
 Meu Zoológico| 670|Tenho uma jiboia ...|
 ojatro| 1031|I wanna see what ...|
 Tingle Triggers| 1212|Well shit now Im ...|
Hope For Paws - O...| 1806|when I saw the en...|
Hope For Paws - O...| 2036|Holy crap. That i...|
 Life Story| 2637|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk| 2698|Call the teddy Larry|
+--------------------+------+--------------------+
only showing top 10 rows

In [3]:
# find user with preference of dog and cat
# note: please propose your own approach and rule to label data 
cond = (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%")\
        | df["comment"].like("%my cat%") | df["comment"].like("%I have a cat%"))

df_clean = df.withColumn('dog_cat',  cond)
df_clean = df_clean.withColumn('has_cat',  (df["comment"].like("%my cat%") | df["comment"].like("%I have a cat%")))
df_clean = df_clean.withColumn('has_dog',  (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%")))
# find user do not have 
df_clean = df_clean.withColumn('no_pet', ~df_clean["comment"].like("%my%") & ~df_clean["comment"].like("%have%")) 


In [4]:
df_clean.show(10)

+--------------------+------+--------------------+-------+-------+-------+------+
 creator_name|userid| comment|dog_cat|has_cat|has_dog|no_pet|
+--------------------+------+--------------------+-------+-------+-------+------+
 Doug The Pug| 87|I shared this to ...| false| false| false| false|
 Doug The Pug| 87| Super cute 😀🐕🐶| false| false| false| true|
 bulletproof| 530|stop saying get e...| false| false| false| false|
 Meu Zoológico| 670|Tenho uma jiboia ...| false| false| false| true|
 ojatro| 1031|I wanna see what ...| false| false| false| true|
 Tingle Triggers| 1212|Well shit now Im ...| false| false| false| true|
Hope For Paws - O...| 1806|when I saw the en...| false| false| false| true|
Hope For Paws - O...| 2036|Holy crap. That i...| false| false| false| true|
 Life Story| 2637|武器はクエストで貰えるんじゃないん...| false| false| false| true|
 Brian Barczyk| 2698|Call the teddy Larry| false| false| false| true|
+--------------------+------+--------------------+-------+-------+-------+------+
only showing top 10 rows

In [5]:
# data preprocessing 
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="text", pattern="\\W")
df_clean = regexTokenizer.transform(df_clean)
df_clean.show(10)

+--------------------+------+--------------------+-------+-------+-------+------+--------------------+
 creator_name|userid| comment|dog_cat|has_cat|has_dog|no_pet| text|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+
 Doug The Pug| 87|I shared this to ...| false| false| false| false|[i, shared, this,...|
 Doug The Pug| 87| Super cute 😀🐕🐶| false| false| false| true| [super, cute]|
 bulletproof| 530|stop saying get e...| false| false| false| false|[stop, saying, ge...|
 Meu Zoológico| 670|Tenho uma jiboia ...| false| false| false| true|[tenho, uma, jibo...|
 ojatro| 1031|I wanna see what ...| false| false| false| true|[i, wanna, see, w...|
 Tingle Triggers| 1212|Well shit now Im ...| false| false| false| true|[well, shit, now,...|
Hope For Paws - O...| 1806|when I saw the en...| false| false| false| true|[when, i, saw, th...|
Hope For Paws - O...| 2036|Holy crap. That i...| false| false| false| true|[holy, crap, that...|
 Life Story| 2637|武器はクエストで貰えるんじゃないん...| false| false| false| true| []|
 Brian Barczyk| 2698|Call the teddy Larry| false| false| false| true|[call, the, teddy...|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+
only showing top 10 rows

#### 1. Build the classifier 
In order to train a model against the comments, you can use RegexTokenizer to split each comment into a list of words and then use Word2Vec or other model to convert the list to a word vector. What Word2Vec does is to map each word to a unique fixed-size vector and then transform each document into a vector using the average of all words in the document.

In [7]:
from pyspark.ml.feature import Word2Vec
# Learn a mapping from words to Vectors. referenced at: https://spark.apache.org/docs/2.1.0/ml-features.html#word2vec
word2Vec = Word2Vec(vectorSize=300, minCount=2, inputCol="text", outputCol="w2v_result") 
# choose vectorSize as 300 a normal practice and minCount=2 based on the 'text' column generated from above cell 
df_word = spark.createDataFrame(df_clean.select("text").take(9960)) 
# Explanation for 9960:for some reason, after 9960, there will be an error, so I choose to run the data set up until 9960 first.

model = word2Vec.fit(df_word) # train the model 
result = model.transform(df_clean) # apply trained model to entire dataset df_clean 
for row in result.take(10):
    creator_name,userid,comment,dog_cat,has_cat,has_dog,no_pet,text,w2v_result = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(w2v_result)))

Text: [i, shared, this, to, my, friends, and, mom, the, were, lol] => 
Vector: [0.0663369093249,-0.063045622601,0.0217102658803,0.0116574961099,-0.00541100701362,-0.0176824128774,-0.0606965498353,-0.00718819697134,-0.00966431524086,0.0250976275843,-0.0205572937875,0.00149407436203,0.00144850165022,0.00195717718452,0.0181580212348,-0.00200145772066,-0.0143261266401,0.00858712688469,0.0143441718537,-0.038177352949,-0.0208552163306,-0.0275454742739,-0.0138060404834,0.00609095150668,-0.050582873926,-0.0144586492838,-0.028127265893,0.0415271504951,0.000221546210179,0.0152011550381,-0.0305521771396,0.00397948906969,0.000791170311541,-0.0373964702568,0.0307814117881,-0.023098704638,-0.0137908152326,0.0435481188734,0.0128938119107,0.0323216981329,0.0010590471449,-0.0220282910847,0.0196609247095,-0.00694109797901,0.0302073968183,-0.0307983367714,-0.00299846810627,0.0256126875703,-0.0105944277073,-0.0378732184922,0.00286207645497,-0.0244364191312,-0.0461155451538,-0.060951012663,0.00101414231837,-0.064944539706,-0.00477774976224,-0.0123180796509,0.00555962196086,-0.0180862640369,0.032062468828,-0.0131378733151,0.0342359053585,-0.0348617492154,0.00865825891114,-0.023050362067,0.0363906190773,-0.0108542738228,0.0206201691154,-0.0024893367028,-0.0479497539557,-0.026360665829,0.0462211982813,0.00436091379114,0.0262808089134,-0.00431026289748,0.0520330534571,-0.0696845047284,-0.0153321649519,-0.0393363487886,0.0142411486172,-0.0330657914747,-0.00113820817999,0.117307922364,-0.0194089797865,0.000617332185026,0.0102383873044,0.00127948202531,-0.0142424545074,0.0140291980298,-0.00438760670791,-0.00255027555712,-0.0291909312787,-0.00909547677094,-0.0519816252477,-0.0155619134682,0.0232110721038,-0.0178175724577,0.00357658489057,-0.0325541325226,0.0406285435099,0.00324915114536,0.0238559403436,-0.0202465375055,-0.0268831214317,-0.0152272853514,0.0324094448641,-0.0408543572174,0.0255667498886,0.0845176763409,-0.00604948246407,-0.000535647162575,-0.00503025535198,0.0233884733607,-0.0345476010222,-0.00514242391695,0.0192167199196,0.026992070859,0.0638072471126,-0.0311939641251,-0.0229547041765,0.00204397439533,0.0391652152625,0.0364185153634,-0.041301506254,0.00540095106011,0.0248320981733,0.0114607485329,0.0353436137666,0.0411251757971,-0.0122008141321,0.0279580615408,-0.00732569728809,0.00170717512215,-0.0031710856446,-0.00860795483459,-0.0170734963507,0.0102433494176,0.0155346337071,0.015655732883,-0.000681157036028,0.0302646278869,-0.048490063835,-0.0133776080858,-0.0421038658612,-0.012683364695,-0.0250438159961,0.0220866928205,-0.0327094198384,-0.0352986954332,0.0378123486298,-0.0114835719525,-0.0158511043674,0.00952572345109,-0.0204621313367,-0.0200451209252,0.0554960167342,-0.0401763782456,-0.0463656859568,-0.0167126905448,-0.0435757351595,0.00555329980456,0.0109175147637,0.00843289381274,-0.00451512745746,0.0392601900014,0.0369524965539,0.0357790118062,-0.0205297815661,0.0292891087751,-0.0171805746557,-0.010557027621,0.0764878733158,0.0135851164391,-0.0341173215155,-0.0396205311091,0.0115838623288,-0.00871057825332,-0.0030008397589,0.0669653403509,-0.0364340729747,-0.0164559808962,-0.013163810871,-0.0133859264633,0.0308963626581,-0.00322691728467,-0.0492042772027,-0.0283537166066,0.0123923254039,0.0296168375168,0.0260973934518,0.0377355681584,-0.0734096705829,-0.055241664226,-0.0202885213393,-0.0267345750789,0.0372811622642,0.0332026395441,0.0206046645445,-0.00660669420507,0.0945682681631,-0.031714813244,0.0637823889159,0.0178872270044,0.0176878033037,-0.031902924651,0.00325237566986,-0.00803441998803,0.0368773985908,-0.0344090147727,-0.0167981938099,0.0565519572701,0.0503132450394,-0.00330035299571,0.0118620028899,0.00549241139511,0.0211969610431,-0.0299040876138,-0.00254330644384,0.0446108871573,-0.00854027550668,-0.00718752373095,0.0121608073585,-0.000620860257186,-0.0128255116446,-0.0161102391536,-0.0140918919351,0.0556058953179,0.00444477686489,0.000760360753206,0.00591941128633,0.0182911816069,-0.00298735965043,-0.0479454256129,0.02504

In [8]:
from pyspark.ml.classification import LogisticRegression
df_clean_train = result.where("has_cat=true or has_dog=true")
df_clean_train.show(10)
df_clean_train.registerTempTable('df_clean_train')
#df_clean_train.createOrReplaceTempView("df_clean_train")
df_clean_train = spark.sql("SELECT CASE when has_cat =true then 1 else 0 end as label, w2v_result  AS features FROM df_clean_train") # create features table for model 
df_clean_train.show(10)
# train LR model
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(df_clean_train)
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 100  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.0001, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(df_clean_train, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 creator_name|userid| comment|dog_cat|has_cat|has_dog|no_pet| text| w2v_result|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 The Dodo| 24401|Now I want to try...| true| false| true| false|[now, i, want, to...|[0.03171658614236...|
 Cole & Marmalade| 43112|I blow smoke in m...| true| true| false| false|[i, blow, smoke, ...|[0.04423690068704...|
Zak Georges Dog T...| 63541|my dog lucky wont...| true| false| true| false|[my, dog, lucky, ...|[0.03052885087206...|
 DarkDynastyK9s|175184|thats what my dog do| true| false| true| false|[thats, what, my,...|[0.03674667021259...|
 The Pet Collective|203881|Im so happy i thi...| true| false| true| false|[im, so, happy, i...|[0.02281381429687...|
 Steff J|273292|Since my cat is g...| true| true| false| false|[since, my, cat, ...|[0.01229083383133...|
 Cole & Marmalade|329273|I have several pl...| true| true| false| false|[i, have, several...|[0.02952577768771...|
Hope For Paws - O...|344406|This is so sad be...| true| false| true| false|[this, is, so, sa...|[0.04237943595753...|
 stacyvlogs|346294|my cat died today...| true| true| false| false|[my, cat, died, t...|[0.03948655641094...|
 Brave Wilderness|381445|my catfish died t...| true| true| false| false|[my, catfish, die...|[0.03300782111182...|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
only showing top 10 rows

+-----+--------------------+
label| features|
+-----+--------------------+
 0|[0.03171658614236...|
 1|[0.04423690068704...|
 0|[0.03052885087206...|
 0|[0.03674667021259...|
 0|[0.02281381429687...|
 1|[0.01229083383133...|
 1|[0.02952577768771...|
 0|[0.04237943595753...|
 1|[0.03948655641094...|
 1|[0.03300782111182...|
+-----+--------------------+
only showing top 10 rows

Model 1 was fit using parameters: 
{Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='maxIter', doc='maximum number of iterations (>= 0)'): 10, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='featuresCol', doc='features column name'): 'features', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='standardization', doc='whether to standardize the training features before fitting the model'): True, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='regParam', doc='regularization parameter (>= 0)'): 0.01, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='fitIntercept', doc='whether to fit an intercept term'): True, Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='labelCol', doc='label column name'): 'label', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5', name='predictionCol', doc='prediction column name'): 'prediction', Param(parent='LogisticRegression_431a8bc4ca874cf5fae5

In [9]:
# test data 
df_clean_valid = result.where("has_cat=true or has_dog=true") # same as train
df_clean_valid.show(10)
df_clean_valid.registerTempTable('df_clean_valid')
df_clean_valid = spark.sql("SELECT userid, CASE when has_cat =true then 1 else 0 end as label, comment, w2v_result AS features FROM df_clean_valid")

# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(df_clean_valid)
valid_result = prediction.select("label","userid", "comment" , "myProbability", "prediction") 

total,correct = 0.,0.

for row in valid_result.collect():
    total += 1
    if row.prediction == row.label:
      correct +=1
print("total accuracy: %d/%d: %.2f" %(correct,total, correct* 100./total))


+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 creator_name|userid| comment|dog_cat|has_cat|has_dog|no_pet| text| w2v_result|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 The Dodo| 24401|Now I want to try...| true| false| true| false|[now, i, want, to...|[0.03171658614236...|
 Cole & Marmalade| 43112|I blow smoke in m...| true| true| false| false|[i, blow, smoke, ...|[0.04423690068704...|
Zak Georges Dog T...| 63541|my dog lucky wont...| true| false| true| false|[my, dog, lucky, ...|[0.03052885087206...|
 DarkDynastyK9s|175184|thats what my dog do| true| false| true| false|[thats, what, my,...|[0.03674667021259...|
 The Pet Collective|203881|Im so happy i thi...| true| false| true| false|[im, so, happy, i...|[0.02281381429687...|
 Steff J|273292|Since my cat is g...| true| true| false| false|[since, my, cat, ...|[0.01229083383133...|
 Cole & Marmalade|329273|I have several pl...| true| true| false| false|[i, have, several...|[0.02952577768771...|
Hope For Paws - O...|344406|This is so sad be...| true| false| true| false|[this, is, so, sa...|[0.04237943595753...|
 stacyvlogs|346294|my cat died today...| true| true| false| false|[my, cat, died, t...|[0.03948655641094...|
 Brave Wilderness|381445|my catfish died t...| true| true| false| false|[my, catfish, die...|[0.03300782111182...|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
only showing top 10 rows

total accuracy: 32404/37462: 86.50

#### 2. Classify All The Users
We can now apply the cat/dog classifiers to all the other users in the dataset.

In [11]:
# Now expanding the trained model to other users in the dataset, specifically the users that were previously classified as having no cat nor dog. 
df_clean_test = result.where("has_cat=false AND has_dog=false")
df_clean_test.show(10)
df_clean_test.registerTempTable('df_clean_test')
df_clean_test = spark.sql("SELECT creator_name, userid, comment, w2v_result AS features FROM df_clean_train")
prediction = model2.transform(df_clean_test)
test_result = prediction.select("creator_name","userid", "comment" , "myProbability", "prediction")
#referenced at: https://spark.apache.org/docs/latest/ml-pipeline.html#example-estimator-transformer-and-param

+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 creator_name|userid| comment|dog_cat|has_cat|has_dog|no_pet| text| w2v_result|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
 Doug The Pug| 87|I shared this to ...| false| false| false| false|[i, shared, this,...|[0.06633690932490...|
 Doug The Pug| 87| Super cute 😀🐕🐶| false| false| false| true| [super, cute]|[0.02996694948524...|
 bulletproof| 530|stop saying get e...| false| false| false| false|[stop, saying, ge...|[0.02020143984753...|
 Meu Zoológico| 670|Tenho uma jiboia ...| false| false| false| true|[tenho, uma, jibo...|[0.00123784136182...|
 ojatro| 1031|I wanna see what ...| false| false| false| true|[i, wanna, see, w...|[0.03245348008137...|
 Tingle Triggers| 1212|Well shit now Im ...| false| false| false| true|[well, shit, now,...|[0.00868930344004...|
Hope For Paws - O...| 1806|when I saw the en...| false| false| false| true|[when, i, saw, th...|[0.02720558880865...|
Hope For Paws - O...| 2036|Holy crap. That i...| false| false| false| true|[holy, crap, that...|[0.02190167517759...|
 Life Story| 2637|武器はクエストで貰えるんじゃないん...| false| false| false| true| []| (300,[],[])|
 Brian Barczyk| 2698|Call the teddy Larry| false| false| false| true|[call, the, teddy...|[0.02779189661669...|
+--------------------+------+--------------------+-------+-------+-------+------+--------------------+--------------------+
only showing top 10 rows

In [12]:
# visualize results
test_result.registerTempTable('test_result')
test_result = spark.sql("SELECT creator_name,userid, comment, CASE WHEN prediction < 0.5 then 'dog_person' ELSE 'cat_person' END as prediction FROM test_result")
test_result.show(20)

+--------------------+------+--------------------+----------+
 creator_name|userid| comment|prediction|
+--------------------+------+--------------------+----------+
 The Dodo| 24401|Now I want to try...|dog_person|
 Cole & Marmalade| 43112|I blow smoke in m...|cat_person|
Zak Georges Dog T...| 63541|my dog lucky wont...|dog_person|
 DarkDynastyK9s|175184|thats what my dog do|dog_person|
 The Pet Collective|203881|Im so happy i thi...|dog_person|
 Steff J|273292|Since my cat is g...|cat_person|
 Cole & Marmalade|329273|I have several pl...|cat_person|
Hope For Paws - O...|344406|This is so sad be...|dog_person|
 stacyvlogs|346294|my cat died today...|cat_person|
 Brave Wilderness|381445|my catfish died t...|cat_person|
 Robin Seplut|424882|The white cat loo...|cat_person|
 Talking Kitty Cat|497349|Random kitty look...|cat_person|
 Mr. Max T.V.|497611|Whenever I play a...|dog_person|
World Animal Awar...|542233|What worries me ...|dog_person|
 Funny Animals|578367|I also faced so m...|cat_person|
 MaxluvsMya|598125|The little black ...|cat_person|
 Gohan The Husky|689464|Am I the feeling ...|dog_person|
 Cole & Marmalade|711033|Maybe not ghosts ...|cat_person|
Hope For Paws - O...|769264|I want all the do...|dog_person|
Gone to the Snow ...|829219|My golden retriev...|dog_person|
+--------------------+------+--------------------+----------+
only showing top 20 rows

#### 3. Get insigts of Users

In [14]:
print("if the user mentions \"dog\", he/she tends to be a dog owner, maked as 0")
print("if the user mentions \"cat\", he/she tends to be a cat owner,maked as 1")
test_result2 = spark.sql("SELECT COUNT(userid), prediction FROM test_result GROUP BY prediction")
test_result2.show()

if the user mentions "dog", he/she tends to be a dog owner, maked as 0
if the user mentions "cat", he/she tends to be a cat owner,maked as 1
+-------------+----------+
count(userid)|prediction|
+-------------+----------+
 23830| 0.0|
 13632| 1.0|
+-------------+----------+

#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [16]:
print("all users are analysed already from the table")
test_result = spark.sql("SELECT creator_name, sum(dog_count), sum(cat_count) FROM \
                        (SELECT creator_name,userid, comment, CASE WHEN prediction < 0.5 then 1 ELSE 0 END as dog_count, \
                                  CASE WHEN prediction >= 0.5 then 1 ELSE 0 END as cat_count FROM test_result) as t \
                                  GROUP BY creator_name")
test_result.show(20)

all users are analysed already from the table
+--------------------+--------------+--------------+
 creator_name|sum(dog_count)|sum(cat_count)|
+--------------------+--------------+--------------+
 Parrot Wizard| 1| 1|
 Jonathan Seabolt| 0| 1|
 The Louisiana SPCA| 1| 0|
 Loki the Red Fox| 102| 42|
 Veronica Morris| 2| 1|
 Keystone Puppies| 2| 0|
 FROSTY Life| 84| 72|
 Aaron Rift| 3| 12|
 Shamama Hunting Owl| 2| 1|
 Miss Aww| 4| 7|
 Ashley Siemon| 1| 0|
 The Experience| 1| 0|
 Clints Reptiles| 0| 2|
Renaissance MarineTV| 1| 0|
 TeamSquatchinUSA| 2| 0|
 Dee Timmy-Hutch-Fan| 2| 0|
 Frozen Kitten| 17| 83|
 Ty The Dog Guy| 5| 1|
 ASPCA| 1| 0|
 Bronwyne Mirkovich| 1| 0|
+--------------------+--------------+--------------+
only showing top 20 rows

#### 5. Analysis and Future work

In [18]:
# try random forest or other machine learning models.